In [1]:
import sagemaker
import boto3 # to be able to interact with AWS services such as S3 for data storage (e.g. reading from s3 bucket)
"""
from sagemaker.amazon.amazon_estimator import get_image_uri # this is used to retrieve the URI (Uniform Resource Identifier) ...
                                                            # ... for a specific built-in algorithm container image provided ...
                                                            # ... by Amazon
"""
from sagemaker import image_uris # new version of get_image_uri
from sagemaker.session import s3_input, Session # if we want to use instances of sagemaker, we need to have session


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# we can create the buckets form here:
bucket_name = 'mlnovo' # Change it to the name of bucket you have created in AWS S3
my_region = boto3.session.Session().region_name # based on the region, we can create different buckets, folders, ...
print(my_region)

eu-central-1


In [3]:
"""
# The next step is to create the bucket name based on the region
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e: 
    print('S3 error:', e)
"""

"\n# The next step is to create the bucket name based on the region\ns3 = boto3.resource('s3')\ntry:\n    if my_region == 'us-east-1':\n        s3.create_bucket(Bucket=bucket_name)\n    print('S3 bucket created successfully')\nexcept Exception as e: \n    print('S3 error:', e)\n"

In [4]:
# set the output path where the trained model is saved
prefix = 'xgboost-as-a-built-in_alg'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://mlnovo/xgboost-as-a-built-in_alg/output


In [5]:
# Downloading the dataset and storing it in S3
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [6]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
# uploading the CSV file into S3 bucket
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
# Reading the training data from S3 bucket and loading it to sagemaker as the "input train data sample"
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

# Loading Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [8]:
# Building models "XGBoost in built alg"
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
# Whenever we use an in-built model from sagemaker, we have to load a container and we do it using get_image_uri

print(boto3.Session().region_name)

container = image_uris.retrieve('xgboost', boto3.Session().region_name, version="latest")

eu-central-1


In [9]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [10]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [11]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: xgboost-2024-03-05-14-00-52-306


2024-03-05 14:00:52 Starting - Starting the training job...
2024-03-05 14:01:13 Starting - Preparing the instances for training......
2024-03-05 14:02:21 Downloading - Downloading input data...
2024-03-05 14:02:51 Downloading - Downloading the training image...
2024-03-05 14:03:12 Training - Training image download completed. Training in progress...Arguments: train
[2024-03-05:14:03:30:INFO] Running standalone xgboost training.
[2024-03-05:14:03:30:INFO] File size need to be processed in the node: 4.83mb. Available memory size in the node: 23948.5mb
[2024-03-05:14:03:30:INFO] Determined delimiter of CSV input is ','
[14:03:30] S3DistributionType set as FullyReplicated
[14:03:30] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-03-05:14:03:30:INFO] Determined delimiter of CSV input is ','
[14:03:30] S3DistributionType set as FullyReplicated
[14:03:30] 12357x59 matrix with 729063 entries loaded from /opt/ml/input/data/v

In [12]:
# Deploying ML model as endpoint
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-03-05-14-09-47-656
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-03-05-14-09-47-656
INFO:sagemaker:Creating endpoint with name xgboost-2024-03-05-14-09-47-656


-----!

In [15]:
# Predcition of the test data
from sagemaker import serializers
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = serializers.CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [16]:
predictions_array

array([0.05578489, 0.04585116, 0.05088526, ..., 0.03450042, 0.03321051,
       0.03635123])

In [ ]:

cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

In [ ]:
# Deleting the endpoints
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()